############################ Phase 1

In [1]:
############################ Phase 1, step 1
############################ Import libraries
import pandas as pd
import numpy as np
import math
import os

In [2]:
############################ Phase 1, step 2
############################ Data loading
path_ = './'
data = pd.read_csv(os.path.join(path_, 'monthly_featuers.csv'))
data

,year,month,lat,lon,soil moisture (mean),precipitation,temperature,ndvi_dif,ndvi
0,2017,4,-0.5,-47.0,0.574741,398.456508,26.572355,0.048321,0.638990
1,2017,5,-0.5,-47.0,0.560529,58.895768,27.313396,0.004121,0.643111
2,2017,6,-0.5,-47.0,0.574662,111.196667,27.196434,-0.022629,0.620482
3,2017,7,-0.5,-47.0,0.571827,37.259126,27.205609,-0.044842,0.575640
4,2017,8,-0.5,-47.0,0.571412,0.000000,27.283456,-0.033857,0.541783
...,...,...,...,...,...,...,...,...,...
1009484,2020,2,9.5,99.0,0.492574,90.488580,26.071872,0.025299,0.756026
1009485,2020,3,9.5,99.0,0.490852,18.020527,27.458084,0.009107,0.765133
1009486,2020,4,9.5,99.0,0.493698,229.356772,27.952469,-0.076257,0.688876
1009487,2020,5,9.5,99.0,0.493269,210.775003,29.398903,0.075696,0.764572


In [3]:
############################ Phase 1, step 3
############################ Define the train and test datasets
mask_train = (data['year'].between(2017,2018)) | ((data['year'] == 2019) & data['month'].between(1,6))
mask_test = (data['year'] == 2020) | ((data['year'] == 2019) & data['month'].between(7,12))

data_test0 = data.loc[mask_test]
data_train0 = data.loc[mask_train]

print('length of train', len(data_train0))
print('length of test', len(data_test0))


length of train 684846
length of test 324643


In [4]:
target = ['ndvi']
featuers = ['precipitation', 'soil moisture (mean)', 'temperature', 'ndvi_dif', 'lat', 'lon', 'month'] 

data_train = data_train0.drop(['year'], axis=1)
data_test = data_test0.drop(['year'], axis=1)

x_train, y_train = data_train.drop(target, axis=1), data_train[target]
x_test, y_test = data_test.drop(target, axis=1), data_test[target]

In [5]:
############################ Phase 1, step 4
############################ Normalizing datasets
from sklearn.preprocessing import MinMaxScaler

def lofa_mm_scaler(x, scaled_typ):
    if (scaled_typ == 'min_max'):
        x_scaled = pd.DataFrame(MinMaxScaler().fit_transform(x), columns = x.columns)
    elif (scaled_typ == 'standrd'):
        x_scaled = pd.DataFrame(StandardScaler().fit_transform(x), columns = x.columns)
    print(len(x_scaled))
    return x_scaled


x_train_scaled = lofa_mm_scaler(x_train, 'min_max')
x_test_scaled = lofa_mm_scaler(x_test, 'min_max')

684846
324643


############################ Phase 2

In [6]:
############################ Phase 2, step 1
############################ Import libraries
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras import Sequential
from tensorflow.keras import Model
import tensorflow as tf

2023-10-05 11:48:40.009156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
############################ Phase 2, step 2
############################ Model details
hidden_units1 = 150
hidden_units2 = 125
hidden_units3 = 100
hidden_units4 = 75
hidden_units5 = 50
hidden_units6 = 25

learning_rate = 0.00001
drop = 0.2
kernel_initi ='random_normal'
l2_reg = 0.01

############################ Model structure
def build_model_using_sequential():
    model = Sequential([
        Dense(hidden_units1, kernel_initializer=kernel_initi, activity_regularizer=regularizers.l2(l2_reg), activation='relu'),
        Dropout(drop),
        Dense(hidden_units2, kernel_initializer=kernel_initi, activity_regularizer=regularizers.L2(l2_reg),activation='relu'),
        Dropout(drop),
        Dense(hidden_units3, kernel_initializer=kernel_initi, activity_regularizer=regularizers.L2(l2_reg),activation='relu'),
        Dropout(drop),
        Dense(hidden_units4, kernel_initializer=kernel_initi, activity_regularizer=regularizers.L2(l2_reg),activation='relu'),
        Dropout(drop),
        Dense(hidden_units5, kernel_initializer=kernel_initi, activity_regularizer=regularizers.L2(l2_reg),activation='relu'),
        Dropout(drop),
        Dense(hidden_units6, kernel_initializer=kernel_initi, activity_regularizer=regularizers.L2(l2_reg),activation='relu'),
        Dropout(drop),
        Dense(1, kernel_initializer=kernel_initi, activation='linear')
        ])
    return model

############################ Model building
model = build_model_using_sequential()



2023-10-05 11:48:45.479333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 11:48:45.480826: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [8]:
############################ Phase 2, step 3
############################ Loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss = msle, 
    optimizer = Adam(learning_rate = learning_rate), 
    metrics = [msle]
)


In [9]:
############################ Phase 2, step 4
############################ Train the model
history = model.fit(
    x_train_scaled.values, 
    y_train.values, 
    epochs = 14,# 14 
    batch_size = 64,
    validation_split = 0.1
)

Epoch 1/14
9631/9631 [==============================] - 86s 9ms/step - loss: 0.0425 - mean_squared_logarithmic_error: 0.0297 - val_loss: 0.0181 - val_mean_squared_logarithmic_error: 0.0095
Epoch 2/14
9631/9631 [==============================] - 78s 8ms/step - loss: 0.0119 - mean_squared_logarithmic_error: 0.0079 - val_loss: 0.0113 - val_mean_squared_logarithmic_error: 0.0083
Epoch 3/14
9631/9631 [==============================] - 81s 8ms/step - loss: 0.0087 - mean_squared_logarithmic_error: 0.0068 - val_loss: 0.0101 - val_mean_squared_logarithmic_error: 0.0085
Epoch 4/14
9631/9631 [==============================] - 79s 8ms/step - loss: 0.0075 - mean_squared_logarithmic_error: 0.0063 - val_loss: 0.0093 - val_mean_squared_logarithmic_error: 0.0081
Epoch 5/14
9631/9631 [==============================] - 80s 8ms/step - loss: 0.0069 - mean_squared_logarithmic_error: 0.0060 - val_loss: 0.0087 - val_mean_squared_logarithmic_error: 0.0078
Epoch 6/14
9631/9631 [==============================] -

In [10]:
############################ Phase 2, step 5
############################ Model Evaluation
from sklearn.metrics import r2_score 

x_test_prediction = model.predict(x_test_scaled)
x_train_prediction = model.predict(x_train_scaled)

train_r2 = r2_score(y_train, x_train_prediction)
test_r2 = r2_score(y_test, x_test_prediction)

print('r2 for train is: ', round(train_r2, 2))
print('r2 for test is: ', round(test_r2, 2))

21402/21402 [==============================] - 110s 5ms/step
r2 for train is:  0.87
r2 for test is:  0.86
